# SQL Parser using Lark
### Lark
- Ref: [Lark Documentation](https://lark-parser.readthedocs.io/en/latest/)
  - Repo: [lark-parser](https://github.com/lark-parser/lark)

### Lark SQL Parser
- Ref: [sql_to_ibis](https://github.com/zbrookle/sql_to_ibis)
- Ref: [ibis](https://github.com/ibis-project/ibis)
  - Docs: https://ibis-project.org/docs/dev/

`sql_to_ibis` is a [Python](https://www.python.org/) package that translates SQL syntax into [ibis](https://github.com/ibis-project/ibis) expressions. This provides the capability of using only one SQL dialect to target many different backends.

- Ref: From `sql_to_ibis` project `sql_select_query.py`
- Ref: SQL grammar file <https://github.com/zbrookle/sql_to_ibis/tree/main/sql_to_ibis/grammar>

## Firebase Client

In [1]:
from firesql.firebase import FirebaseClient
from firesql.sql.sql_fire_client import FireSQLClient

firebaseClient = FirebaseClient()
firebaseClient.connect(credentials_json='../credentials/credentials.json')

# create FireSQLClient that is using the FireSQLAbstractClient interface.
client = FireSQLClient(firebaseClient)

In [2]:
import os
from lark import Lark, tree

_ROOT = "../firesql/sql"
GRAMMAR_PATH = os.path.join(_ROOT, "grammar", "firesql.lark")
with open(file=GRAMMAR_PATH) as sql_grammar_file:
    _GRAMMAR_TEXT = sql_grammar_file.read()
parser = Lark(_GRAMMAR_TEXT, parser="lalr")

## SQL Insert

In [3]:
from firesql.sql.sql_transformer import SQLTransformer

company = 'bennycorp'

sql1 = """
  INSERT INTO Companies/{}/Users (email, state)
    VALUES ('btscheung+test1@gmail.com', 'ACTIVE')
""".format(company)

ast = parser.parse(sql1)
statement = SQLTransformer().transform(ast)
statement

SQL_Insert(table=SQL_SelectFrom(part='Companies/bennycorp/Users', alias=None), columns=[SQL_ColumnRef(table=None, column='email', func=None), SQL_ColumnRef(table=None, column='state', func=None)], values=[SQL_ValueString(value='btscheung+test1@gmail.com'), SQL_ValueString(value='ACTIVE')])

In [4]:
from firesql.sql import SQLFireInsert

sqlCommand = SQLFireInsert()
sqlCommand.generate(statement, options={})

True

In [5]:
print(sqlCommand.part)
print(sqlCommand.columns)
print(sqlCommand.values)

Companies/bennycorp/Users
['email', 'state']
['btscheung+test1@gmail.com', 'ACTIVE']


In [6]:
document = sqlCommand.build()
print(document)

{'email': 'btscheung+test1@gmail.com', 'state': 'ACTIVE'}


In [7]:
insertedDoc = sqlCommand.execute(client, document)
insertedDoc

{'email': 'btscheung+test1@gmail.com',
 'state': 'ACTIVE',
 'docid': 'EDhlpEHvaHLi7rZSqxcb'}

## SQL Update

In [10]:
from firesql.sql.sql_transformer import SQLTransformer

company = 'bennycorp'

sql1 = """
  UPDATE Companies/{}/Users
  SET state = 'INACTIVE'
  WHERE state = 'ACTIVE' and email = 'btscheung+test1@gmail.com'
""".format(company)

ast = parser.parse(sql1)
statement = SQLTransformer().transform(ast)
statement

SQL_Update(table=SQL_SelectFrom(part='Companies/bennycorp/Users', alias=None), sets=[SQL_BinaryExpression(operator='==', left=SQL_ColumnRef(table=None, column='state', func=None), right=SQL_ValueString(value='INACTIVE'))], where=SQL_BinaryExpression(operator='and', left=SQL_BinaryExpression(operator='==', left=SQL_ColumnRef(table=None, column='state', func=None), right=SQL_ValueString(value='ACTIVE')), right=SQL_BinaryExpression(operator='==', left=SQL_ColumnRef(table=None, column='email', func=None), right=SQL_ValueString(value='btscheung+test1@gmail.com'))))

In [11]:
from firesql.sql import SQLFireUpdate  

sqlCommand = SQLFireUpdate()
queries = sqlCommand.generate(statement, options={})

In [12]:
fireQueries = sqlCommand.firebase_queries(queries)
fireQueries

{'Companies/bennycorp/Users': [['state', '==', 'ACTIVE'],
  ['email', '==', 'btscheung+test1@gmail.com']]}

In [13]:
filterQueries = sqlCommand.filter_queries(queries)
filterQueries

{'Companies/bennycorp/Users': []}

In [14]:
documents = sqlCommand.execute(client, fireQueries)

In [15]:
filterDocs = sqlCommand.filter_documents(documents, filterQueries)
filterDocs

{'Companies/bennycorp/Users': {'EDhlpEHvaHLi7rZSqxcb': {'email': 'btscheung+test1@gmail.com',
   'state': 'ACTIVE',
   'events': [{'event': 'CREATION',
     'date': DatetimeWithNanoseconds(2022, 4, 22, 13, 12, 48, 338000, tzinfo=datetime.timezone.utc)}]}}}

In [16]:
selectDocs = sqlCommand.update_post_process(filterDocs)

In [17]:
sqlCommand.select_fields()

['docid', 'email', 'events', 'state']

In [18]:
from firesql.sql import DocPrinter

docPrinter = DocPrinter()
docPrinter.printCSV(selectDocs, sqlCommand.select_fields())

"docid","email","events","state"
"EDhlpEHvaHLi7rZSqxcb","btscheung+test1@gmail.com","{\"event\": \"CREATION\", \"date\": \"2022-04-22T13:12:48\"}","INACTIVE"


In [19]:
updatedDocs = sqlCommand.update_execute(client, filterDocs)

## SQL Delete

In [20]:
from firesql.sql.sql_transformer import SQLTransformer

company = 'bennycorp'

sql1 = """
  DELETE FROM Companies/{}/Users
    WHERE 
      email = 'btscheung+test1@gmail.com'
""".format(company)

ast = parser.parse(sql1)
statement = SQLTransformer().transform(ast)
statement

SQL_Delete(table=SQL_SelectFrom(part='Companies/bennycorp/Users', alias=None), where=SQL_BinaryExpression(operator='==', left=SQL_ColumnRef(table=None, column='email', func=None), right=SQL_ValueString(value='btscheung+test1@gmail.com')))

In [21]:
from firesql.sql import SQLFireDelete

sqlCommand = SQLFireDelete()
queries = sqlCommand.generate(statement, options={})


In [22]:
print(sqlCommand.collections)
print(queries)

{'Companies/bennycorp/Users': 'Companies/bennycorp/Users'}
{'Companies/bennycorp/Users': [['email', '==', 'btscheung+test1@gmail.com']]}


In [23]:
fireQueries = sqlCommand.firebase_queries(queries)
fireQueries

{'Companies/bennycorp/Users': [['email', '==', 'btscheung+test1@gmail.com']]}

In [24]:
filterQueries = sqlCommand.filter_queries(queries)
filterQueries

{'Companies/bennycorp/Users': []}

In [25]:
documents = sqlCommand.execute(client, fireQueries)

In [26]:
filterDocs = sqlCommand.filter_documents(documents, filterQueries)
filterDocs

{'Companies/bennycorp/Users': {'EDhlpEHvaHLi7rZSqxcb': {'email': 'btscheung+test1@gmail.com',
   'events': [{'event': 'CREATION',
     'date': DatetimeWithNanoseconds(2022, 4, 22, 13, 12, 48, 338000, tzinfo=datetime.timezone.utc)}],
   'state': 'INACTIVE'}}}

In [27]:
selectDocs = sqlCommand.delete_post_process(filterDocs)

In [28]:
sqlCommand.select_fields()

['docid', 'email', 'events', 'state']

In [29]:
from firesql.sql import DocPrinter

docPrinter = DocPrinter()
docPrinter.printCSV(selectDocs, sqlCommand.select_fields())

"docid","email","events","state"
"EDhlpEHvaHLi7rZSqxcb","btscheung+test1@gmail.com","{\"event\": \"CREATION\", \"date\": \"2022-04-22T13:12:48\"}","INACTIVE"


In [30]:
deletedDocs = sqlCommand.delete_execute(client, filterDocs)